# pyBela

## Setup

In [ ]:
import pyBela

# establish communication with bela

bela = pyBela.find_bela()
pyBela.handshake_bela(bela, sleep_time=1, handshake_code=0)

In [ ]:
# http://docs.bela.io/structBelaContext.html variables in context should be accessible from bela.context

sample_rate = bela.context.analogSampleRate
n_analog_channels = bela.context.analogInChannels 
analog_block_size = bela.context.analogFrames 


## Basic GPIO API
For reading and writing values on demand to the GPIO

In [ ]:
from pyBela import gpio

value = gpio.analog_read('A0') # value can also be passed as bitarray '10000000'
gpio.digital_write('D2', 0) # set D2 to 0

## Streaming
- Asynchronously sends data to the jupyter notebook. 



In [ ]:
from pyBela import Streammer

# init and run streammer
time_in_samples = 3*sample_rate # 3s
num_blocks = ceil(time_in_samples/analog_block_size)

streammer = Streammer(num_blocks=num_blocks) # if None, it runs until stopped (ctrl+c or button in Bela cape)

streammer.run(plot=True) # Q add callback here too?
# bela cape light turns on to indicate streamming is happening

# dynamic bokeh plot with tabs for sensors and zoom etc. 
# Q the bokeh plot could also have an ipywidget to stop the recording ? -- but going into GUI never ends

In [ ]:
data = streammer.get_data() # (num_blocks*block_size, n_analog_channels)

## Monitoring
- Samples a variable in the code or a channel. 
- Can be used to get a single value (on-request fashion) or a number of blocks
- Variables can be plotted dynamically

In [ ]:
from pyBela import Watcher

# init and run watcher
watcher = Watcher(analog_channels="11100000", # we select analog channels A0, A1 and A2 with a bitarray
                  num_blocks = 16) 


## a callback can be passed to the watcher to do something given a condition in the watched values. e.g., turn a light on in the Bela cape if the signal peaks

def callback(n): # n is the current frame 
    chIdx = 1; # Analog Channel A1
    if watcher.buffer[chIdx][n] > 0.4: # if frame n on channel A1 is above threshold
        lightIdx = 1 # e.g., blue light, red light is reserved for indicating that streamming / watching / logging is running 
        watcher.blinkLight(lightIdx); 
        watcher.callbackRegister[n] = 1; # set flag in callback register -- Q not sure if this make sense if there's no one to clean it 


# run
watcher.run(plot=True, callback=callback)

# Q does it make sense here to have a streamming / dynamic plot with bokeh if the measurements are always gonna be of only a number of ms?
# It probably does make sense to have a sort of Oscilloscope plot ? -- is the oscilloscope streamming or monitoriing?
# maybe if you want an oscilloscope it is better to just stream

In [ ]:
from bela.utils import plotter

# retrieve data and plot
data = watcher.get_data() # (analog_block_size*4, analogInChannels)

plotter(data) # bokeh plot with options to show/hide every signal, zooming in, etc.

Watcher can also be used to ask for a single variable reading - same functionality as the basic GPIO API

In [ ]:
watcher = Watcher() 
voltage = watcher.read(channel=0)

## Logging
- The Bela Logger logs data into files that live either in the Bela or in the Host machine. 
- The utility of logging directly into the host machine avoids running out of memory in the Bela for long measurements (e.g., a study, large dataset)
- Files are saved as binary to occupy less space

In [ ]:
from pyBela import Logger

# init and run logger
logger = Logger(where="BELA", # can be in host or bela
                analog_channels="11100000") # Logger automatically logs all analog channels. Channels can be selected with a bitarray. Here we are selecting the analog In channels A0, A1 and A2. 

logger.run(name="study1.log", num_blocks=None) # num_blocks is the number of blocks the logger runs. if None, runs until stopped 
# when this cell is ran -a- light in the Bela board is turned on indicating that the logger is running 
# can be stopped with ctrl-c and ideally also from a button in the Bela cape 

In [ ]:
# transfer logs to host
logger.trans(auto_clean=False) # transfer file to jupyter notebook directory.
logger.clean() # cleans logger files in Bela after transferring them to the host

In [ ]:
# load logged data
logged_data = logger.load() # loads logged data as numpy. the logger object knows how many channels have been recorded and hence can decode the binary file. data can also be loaded directly from the binary file but the number of channels and encoding needs to be known in advance

### to do 
- bela code 
  
### extras
- labelling ?
- widgets ?
- provide official kernel ? (why does CircuitPython provide its own?)